In [1]:
import pandas as pd
from sqlalchemy import create_engine

bd = "sakila"
password = "hari1234"  # Asegúrate de reemplazar esto con tu contraseña real
connection_string = 'mysql+pymysql://root:' + password + '@localhost/' + bd
engine = create_engine(connection_string)

In [2]:
def rentals_month(engine, month, year):
    # Create a SQL query that retrieves rental data for the specified month and year
    query = f"""
        SELECT rental_id, rental_date, customer_id
        FROM rental
        WHERE MONTH(rental_date) = {month} AND YEAR(rental_date) = {year};
    """
   
    df = pd.read_sql(query, engine)
    return df


In [3]:
def rental_count_month(df, month, year):
    # Group by customer_id and count the number of rentals for each customer
    rental_counts = df.groupby('customer_id')['rental_id'].count().reset_index()
    
    # Rename the rental count column based on the month and year
    rental_counts.rename(columns={'rental_id': f'rentals_{month:02}_{year}'}, inplace=True)
    
    return rental_counts


In [4]:
def compare_rentals(df1, df2):
    # Merge the two DataFrames on customer_id
    merged_df = pd.merge(df1, df2, on='customer_id', how='outer', suffixes=('_1', '_2')).fillna(0)
    
    # Calculate the difference between rentals in the two months
    merged_df['difference'] = merged_df.iloc[:, 1] - merged_df.iloc[:, 2]
    
    return merged_df


In [5]:
# Get rental data for May and June 2005
may_rentals = rentals_month(engine, 5, 2005)
june_rentals = rentals_month(engine, 6, 2005)

# Count rentals for each customer in May and June
may_counts = rental_count_month(may_rentals, 5, 2005)
june_counts = rental_count_month(june_rentals, 6, 2005)

# Compare the number of rentals between May and June
comparison_df = compare_rentals(may_counts, june_counts)

print(comparison_df)


     customer_id  rentals_05_2005  rentals_06_2005  difference
0              1              2.0              7.0        -5.0
1              2              1.0              1.0         0.0
2              3              2.0              4.0        -2.0
3              4              0.0              6.0        -6.0
4              5              3.0              5.0        -2.0
..           ...              ...              ...         ...
593          595              1.0              2.0        -1.0
594          596              6.0              2.0         4.0
595          597              2.0              3.0        -1.0
596          598              0.0              1.0        -1.0
597          599              1.0              4.0        -3.0

[598 rows x 4 columns]
